In [ ]:
import numpy as np 
from matplotlib import pyplot as plt
%matplotlib inline
from astropy.stats import bayesian_blocks
from os.path import expanduser


In [ ]:
home = expanduser('~')
gc_dir = home + "/Dropbox/GalacticCenter/"

<h2> Extract and plot points from lightcurve 

In [ ]:
# set the energy limits 
E = 0 

In [ ]:
# extract the lightcurve data points

lc_data = np.genfromtxt(gc_dir+"/results/lightcurve_SgrA_daily_points_v254.txt") 
#lc_data = np.genfromtxt(gc_dir+"/log/lightCurves/SgrA_lightCurve_both_4tels_all_E"+str(E)+"_annual_points.txt")
#lc_data = np.genfromtxt(gc_dir+"/log/lightCurves/SgrA_lightCurve_both_E"+str(E)+"_annual_points.txt")

lc_data_points = []
for i in range(len(lc_data[:,0])):
    if lc_data[i,3] != 0 and lc_data[i,1] > 0.: # 
        lc_data_points.append(lc_data[i,:])

lc_data_points = np.array(lc_data_points) 


MJD_array = np.array(lc_data_points[:,0], dtype=np.float64)
flux_obs_array = np.array(lc_data_points[:,1], dtype=np.float64) #  / (m^2 * s)
flux_err_array = np.array(lc_data_points[:,2], dtype=np.float64)
livetime_array = np.array(lc_data_points[:,3], dtype=np.float64)

#print(flux_obs_array)
#log = list(map(lambda l: np.log10(l), _array))


In [ ]:
fluxtime_array = flux_obs_array * livetime_array

# find the total livetime and mean flux 
total_livetime = np.sum(livetime_array)
mean_flux = np.sum(fluxtime_array) / total_livetime
print(mean_flux)


In [ ]:
#lc_data = list(filter((0).__ne__, )

#flux_obs_array[ flux_obs_array==0 ] = np.nan
#flux_err_array[ flux_err_array==0 ] = np.nan
#fluxtime_array[ fluxtime_array==0 ] = np.nan

# remove empty points, marked as 0's 
#flux_obs_array = np.array(list(filter((0.0).__ne__, lc_data[:,1])), dtype=np.float64)
#flux_err_array = np.array(list(filter((0.0).__ne__, lc_data[:,2])), dtype=np.float64)
#livetime_array = np.array(list(filter((0.0).__ne__, lc_data[:,3])), dtype=np.float64)


In [ ]:
plt.errorbar(MJD_array, flux_obs_array, yerr=flux_err_array, fmt='b.') # 

plt.title("Sgr A* Integral Flux ("+str(E)+"TeV < E < 100TeV)")
plt.xlabel(r"$time\left(MJD\right)$")
plt.ylabel(r"flux $\frac{gamma}{m^{2}s}$") # gamma / m^2*s*TeV
#plt.xlim(1.5e0, 5.e1)
plt.ylim(ymin=0.)
plt.axhline(mean_flux)

#! MAY NEED TO REPLOT E0?  
#plt.savefig(gc_dir+"/plots/lightCurve/SgrA_lightcurve_all_4tels_seasonal.png")
#plt.savefig(gc_dir+"/plots/lightCurve/SgrA_lightcurve_all_4tels_E"+str(E)+"_seasonal.png")


<h2> Sliding window

In [ ]:
from math import sqrt, erfc

def S_win(Nw, No, alpha):
    """calculates significance of deviation of flux 
    Nw: excess counts inside window
    No: excess counts outside window
    alpha: ratio of window duration to duration outside of window"""
    
    return (Nw - alpha*No)/sqrt(Nw + alpha**2*No)

def P_pre(S):
    """the chance probability calculated from significance for a window"""
    
    return erfc(S/sqrt(2))

def P_post(P_pre, N):
    """post-trials probability after applying trials factors"""

    return 1 - (1-P_pre)**N


In [ ]:
N = len(lc_data_points)
n = 5 # window size 

Ntot = np.sum(fluxtime_array)
Sig_windows = []

for i in range(N):
    Nw = np.sum(fluxtime_array[i:(i+n)])
    No = Ntot - Nw
    Tw = np.sum(livetime_array[i:(i+n)])
    alpha = Tw / (total_livetime-Tw) 
    Sig_windows.append(S_win(Nw, No, alpha)) 
    
#print(Sig_windows)
#sig_hist = plt.hist(Sig_windows)

p_hist, bin_edges = np.histogram(Sig_windows, density=True)
print(bin_edges)
print(p_hist)
        
plt.hist(Sig_windows)


In [ ]:
# fit hist to Gaussian 
from scipy.optimize import curve_fit

def gauss_norm(x, x0, sigma, A=1):
    """normalized 1D gaussian function"""
    return A/(np.sqrt(2*np.pi))*np.exp(-((x-x0)/sigma)**2/2)



In [ ]:
x = (bin_edges[:-1] + bin_edges[1:]) / 2 
print(x)
print(p_hist)
plt.plot(x, p_hist)

In [ ]:
p0 = (-.001, 300, .005)

p_opt, p_cov = curve_fit(gauss_norm, x, p_hist, p0)

print(p_opt)
print(p_cov)

In [ ]:
plt.plot(x, gauss_norm(x, -.001, 300, .005))

In [ ]:
# find negative numbers 
for i in range(len(fluxtime_array)):
    if fluxtime_array[i] < 0:
        print(fluxtime_array[i])

plt.bar(bin_edges[:-1], p_hist, width = 1)
plt.xlim(min(bin_edges), max(bin_edges))
plt.show() 


<h2> Bayesian blocks

In [ ]:
time_array = (MJD_array*100).astype(np.float64)
count_array = (fluxtime_array*10**7).astype(np.float64)

In [ ]:
bayesian_blocks?

In [ ]:
bin_edges = bayesian_blocks(time_array, count_array)